# IE507 Lab 09
### Nonlinear Optimization with Pyomo
### October 06, 2021
(Narayan Rangaraj, Abijith P Y, Abhishek Chaudury)

---

One of the Pyomo's unique features is its ability to model nonlinear optimization problems. Several other Python packages are available for linear models (PuLP, CPLEX, Gurobi). However, Pyomo can be used for modeling general purpose nonlinear problems in Python also. 

Today we will try to model and solve some nonlinear problems. We first need to install a solver for nonlinear optimization. [Ipopt](http://projects.coin-or.org/Ipopt) is an open-source solver for constrained optimization problems. It is pronounced as "I--P--Opt". Like most other nonlinear solvers for continuous problems, it finds only locally minimum points. The solution it finds may not be globally optimal.

Ipopt has been compiled and made available online so that we may download and run it within this notebook. Ipopt also needs some fortran libraries which need to be installed. Let us install them.

In [1]:
%%bash
rm -fv ipopt*
apt install -q -yy libgfortran5
sleep 1
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/ipopt
wget -q https://www.ieor.iitb.ac.in/files/faculty/amahajan/test.nl
chmod u+rx ipopt

removed 'ipopt'
Reading package lists...
Building dependency tree...
Reading state information...
libgfortran5 is already the newest version (8.4.0-1ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


The above code also fetches an optimization problem in an '.nl' format. 'nl' format is suitable for solvers, not for humans. Lets call ipopt directly (without using any Python) to solve it.

In [2]:
!./ipopt test.nl



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      230

Total number of variables............................:       30
                     variables with only lower bounds:       30
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equ


If you are able to see Ipopt output, we can install pyomo as usual

In [3]:
!pip install -q pyomo

In [4]:
import numpy as np

In [5]:
from pyomo.environ import *

Let us try to solve a nonlinear optimization problem without any constraints
$\min \ 100(x_2 - x_1^2)^2 + (1-x_1)^2$. Pyomo uses two asterisks (**) to denote 'raise to power'. Other mathematical operators like sqrt (for square root), log, sin, cos, multiplication and division etc are available.

In [6]:
model = ConcreteModel()
model.x1 = Var()
model.x2 = Var()

model.obj = Objective(expr = 100.0*(model.x2-model.x1**2)**2 + (1-model.x1)**2, sense=minimize)
model.pprint()

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 100.0*(x2 - x1**2)**2 + (1 - x1)**2

3 Declarations: x1 x2 obj



Next load the solver and pass the model to it, solve and get the results, just like you did for linear optimization.

In [7]:
opt_ipopt = SolverFactory('ipopt')
result = opt_ipopt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj())

print('\nDecision Variables')
print('x1 = ', model.x1.value)  
print('x2 = ', model.x2.value)  


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02849268913269043
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  1.3288608467480825e-28

Decision Variables
x1 =  0.9999999999999899
x2 =  0.9999999999999792


Now go to the questions and try to solve them.

In [8]:
model.x1.setlb(5)
model.x1.setub(10)
model.x2.setlb(5)
model.x2.setub(10)

In [9]:
model.pprint()

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     5 : 0.9999999999999899 :    10 : False : False :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     5 : 0.9999999999999792 :    10 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 100.0*(x2 - x1**2)**2 + (1 - x1)**2

3 Declarations: x1 x2 obj


In [10]:
result = opt_ipopt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj())

print('\nDecision Variables')
print('x1 = ', model.x1.value)  
print('x2 = ', model.x2.value) 


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02265477180480957
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  22516.0

Decision Variables
x1 =  5.0
x2 =  10.0


#Ans 2 :

The new optimal value is 22516.

The new values of x1 and x2 are 5 and 10 respectively.

Solver Status is Ok and Termination condition is optimal.

In [11]:
model.constraints = ConstraintList()
model.constraints.add(model.x1**2 - 14*model.x1 + model.x2**2 - 12*model.x2 <= -83)

In [12]:
model.pprint()

1 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     5 :   5.0 :    10 : False : False :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     5 :  10.0 :    10 : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 100.0*(x2 - x1**2)**2 + (1 - x1)**2

1 Constraint Declarations
    constraints : Size=1, Index=constraints_index, Active=True
        Key : Lower : Body                          : Upper : Active
          1 :  -Inf : x1**2 - 14*x1 + x2**2 - 12*x2 : -83.0 :   True

5 Declarations: x1 x2 obj constraints_index constraints


In [13]:
result = opt_ipopt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj())

print('\nDecision Variables')
print('x1 = ', model.x1.value)  
print('x2 = ', model.x2.value) 

print('\nConstraints')
model.constraints.display()


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02224874496459961
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  63212.3297363497

Decision Variables
x1 =  5.591404989462348
x2 =  6.125940169329545

Constraints
constraints : Size=1
    Key : Lower : Body               : Upper
      1 :  None : -82.99999917003768 : -83.0


#Ans 3 :

The new optimal value is 63212.33.

The new values of x1 and x2 are 5.59 and 6.13 respectively.

The constraint added is approximately equal to -83 and hence is active.

Solver Status is Ok and Termination condition is optimal.

In [14]:
model.x1.setlb(None)
model.x1.setub(None)
model.x2.setlb(None)
model.x2.setub(None)

In [15]:
model.pprint()

1 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :  None : 5.591404989462348 :  None : False : False :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :  None : 6.125940169329545 :  None : False : False :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 100.0*(x2 - x1**2)**2 + (1 - x1)**2

1 Constraint Declarations
    constraints : Size=1, Index=constraints_index, Active=True
        Key : Lower : Body                          : Upper : Active
          1 :  -Inf : x1**2 - 14*x1 + x2**2 - 12*x2 : -83.0 :   True

5 Declarations: x1 x2 obj constraints_i

In [16]:
result = opt_ipopt.solve(model)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('\nObjective = ', model.obj())

print('\nDecision Variables')
print('x1 = ', model.x1.value)  
print('x2 = ', model.x2.value) 

print('\nConstraints')
model.constraints.display()


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 2
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.01954507827758789
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal

Objective =  63212.32973700665

Decision Variables
x1 =  5.591404989472647
x2 =  6.125940169314059

Constraints
constraints : Size=1
    Key : Lower : Body               : Upper
      1 :  None : -82.99999917007058 : -83.0


#Ans 4 :

Since in part (3) the bound constraints are not tight, we remove them. We see that the solution does not change.

The optimal value is 63212.33.

The values of x1 and x2 are 5.59 and 6.13 respectively.